# Data Pre-Processing techniques


## Install Dependencies


In [ ]:
%pip install datasets
%pip install transformers
%pip install spacy


BELOW TAKES FOREVER!

In [ ]:
%pip install spacy-transformers
%pip install transformers[torch]
%pip install seqeval

In [36]:
import torch
import torchtext

SEED = 1234
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

print("PyTorch Version: ", torch.__version__)
print("torchtext Version: ", torchtext.__version__)
print(f"Using {'GPU' if str(DEVICE) == 'cuda' else 'CPU'}.")

PyTorch Version:  2.2.1+cu121
torchtext Version:  0.17.1+cpu
Using GPU.


In [37]:
from datasets import load_dataset, load_metric
dataset = load_dataset("surrey-nlp/PLOD-CW")

## Load BERT Model from transformers library

In [5]:
from transformers import AutoTokenizer, AutoModelForTokenClassification

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = AutoModelForTokenClassification.from_pretrained("bert-base-uncased", num_labels=4)

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [46]:
training = dataset["train"]
valid = dataset["validation"]
test = dataset["test"]

training_tokens = training["tokens"]
training_labels = training["ner_tags"]
valid_tokens = valid["tokens"]
valid_labels = valid["ner_tags"]
test_tokens = test["tokens"]
test_labels = test["ner_tags"]



Function for forming ngrams out of tokens and labels


In [7]:
def n_grams(tokens, labels, n):
  all_ngrams = []
  all_labels = []
  for i in range(0,len(tokens)):
    for j in range(len(tokens[i]) - n +1):
      all_ngrams.append(tokens[i][j:j+n])
      all_labels.append(labels[i][j:j+n])
  return all_ngrams, all_labels



Form n-grams for the different dataset splits

In [8]:
training_ngrams, training_ner = n_grams(training_tokens, training_labels, 2)
valid_ngrams, valid_ner = n_grams(valid_tokens, valid_labels, 2)
test_ngrams, test_ner = n_grams(test_tokens, test_labels, 2)
len(training_ngrams), len(valid_ngrams), len(test_ngrams)



(38928, 4874, 4847)

# Need to convert labels to class indexes so that the model can understand them

In [9]:
def conv_label_indexes(training, valid, test ):
	label_encoding = {"B-O": 0, "B-AC": 1, "B-LF": 2, "I-LF": 3}

	label_list = []
	for sample in training:
		label_list.append([label_encoding[tag] for tag in sample])

	val_label_list = []
	for sample in valid:
		val_label_list.append([label_encoding[tag] for tag in sample])

	test_label_list = []
	for sample in test:
		test_label_list.append([label_encoding[tag] for tag in sample])
	return label_list, val_label_list, test_label_list





In [10]:
label_list, val_label_list, test_label_list = conv_label_indexes(training_ner, valid_ner, test_ner)

In [11]:
def tokenize_and_align_labels(ngrams, list_name):
    tokenized_inputs = tokenizer(ngrams, truncation=True, is_split_into_words=True) ## For some models, you may need to set max_length to approximately 500.

    labels = []
    for i, label in enumerate(list_name):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx])
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [12]:
tokenized_datasets = tokenize_and_align_labels(training_ngrams, label_list)
tokenized_val_datasets = tokenize_and_align_labels(valid_ngrams, val_label_list)
tokenized_test_datasets = tokenize_and_align_labels(test_ngrams, test_label_list)
# print(tokenized_datasets)

In [13]:
# BERT's tokenizer returns the dataset in the form of a dictionary of lists (sentences).
# we have to convert it into a list of dictionaries for training.
def turn_dict_to_list_of_dict(d):
    new_list = []

    for labels, inputs in zip(d["labels"], d["input_ids"]):
        entry = {"input_ids": inputs, "labels": labels}
        new_list.append(entry)

    return new_list

In [14]:
tokenised_train = turn_dict_to_list_of_dict(tokenized_datasets)
tokenised_val = turn_dict_to_list_of_dict(tokenized_val_datasets)
tokenised_test = turn_dict_to_list_of_dict(tokenized_test_datasets)

In [15]:
from transformers import DataCollatorForTokenClassification
data_collator = DataCollatorForTokenClassification(tokenizer)

In [16]:
import numpy as np

metric = load_metric("seqeval")
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

<ipython-input-16-7c72ccf86762>:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("seqeval")
/usr/local/lib/python3.10/dist-packages/datasets/load.py:756: FutureWarning: The repository for seqeval contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/seqeval/seqeval.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [17]:
from transformers import TrainingArguments, Trainer, EarlyStoppingCallback

# Training arguments (feel free to play arround with these values)
model_name = "bert-base-uncased"
epochs = 6
batch_size = 4
learning_rate = 2e-5

args = TrainingArguments(
    f"BERT-finetuned-NER",
    # evaluation_strategy = "epoch", ## Instead of focusing on loss and accuracy, we will focus on the F1 score
    evaluation_strategy ='steps',
    eval_steps = 7000,
    save_total_limit = 3,
    learning_rate=learning_rate,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=epochs,
    weight_decay=0.001,
    save_steps=35000,
    metric_for_best_model = 'f1',
    load_best_model_at_end=True
)

trainer = Trainer(
    model,
    args,
    train_dataset=tokenised_train,
    eval_dataset=tokenised_val,
    data_collator = data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks = [EarlyStoppingCallback(early_stopping_patience=3)]
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False)
  warnings.warn(


In [ ]:
trainer.train()

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
7000,0.441500,0.378849,0.871570,0.890230,0.880801,0.931669
14000,0.333200,0.536219,0.887714,0.898599,0.893123,0.935803
21000,0.289800,0.562518,0.895829,0.898793,0.897309,0.936505
28000,0.266400,0.635242,0.887604,0.896069,0.891816,0.928471
35000,0.211900,0.719996,0.896378,0.900740,0.898554,0.931981
42000,0.163900,0.746512,0.893264,0.900740,0.896986,0.933619
49000,0.140000,0.818655,0.897307,0.901323,0.899311,0.938300
56000,0.084600,0.886056,0.891053,0.899377,0.895196,0.933385


/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: [0, 0] seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: [0, 2] seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: [0, 0] seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: [0, 2] seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: [0, 0] seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_label

TrainOutput(global_step=58392, training_loss=0.26773516840110956, metrics={'train_runtime': 7461.3421, 'train_samples_per_second': 31.304, 'train_steps_per_second': 7.826, 'total_flos': 707855506218528.0, 'train_loss': 0.26773516840110956, 'epoch': 6.0})

![image.png](attachment:image.png)

In [71]:
# Prepare the test data for evaluation in the same format as the training data
def calculate_results(trainer, dataset):

	predictions, labels, _ = trainer.predict(tokenised_test)
	predictions = np.argmax(predictions, axis=2)

	# Remove the predictions for the [CLS] and [SEP] tokens
	true_predictions = [
			[label_list[p] for (p, l) in zip(prediction, label) if l != -100]
			for prediction, label in zip(predictions, labels)
	]
	true_labels = [
			[label_list[l] for (p, l) in zip(prediction, label) if l != -100]
			for prediction, label in zip(predictions, labels)
	]

	# Compute multiple metrics on the test restuls
	results = metric.compute(predictions=true_predictions, references=true_labels)
	return results


# Results of 2-gram converted dataset
From the above results we can see that the model performs decently, but not better than just using BERTs regular tokenizer on the regular dataset.
Additionally, by applying n-grams to the dataset, we increase the number of tokens in our dataset by multiple factors. This makes the training process much more computationally expensive. So worse performance for more computation is not a good idea

![alt text](test1.png)

# Analysis of use of n-grams for BERT model
Above we can see the results of using n-grams on the dataset before tokenizing and training the BERT model.
BERT uses the encoder architecture meaning that it looks at tokens in both directions (good at capturing context). For this reason, n-grams introduce a lot of unnecessary / redundant and repeated data.\
It is also likely that the n-grams introduced a lot of repeated data during BERT tokenization process.
The main observation to make is the amount of tokens we end up with as a result of n-grams (much larger than without). This makes model training expensive without actually providing any improvement.



# Data Pre-Processing techniques #2: Removing stopwords
The 2nd approach I am going to take is to remove stopwords from the dataset (both tokens and their associated labels)

In [27]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [28]:
import nltk
from nltk.corpus import stopwords

def remove_stopwords(tokens, labels):
    stop_words = set(stopwords.words('english'))
    filtered_tokens = []
    filtered_labels = []

    for sublist_tokens, sublist_labels in zip(tokens, labels):
        filtered_tokens_sublist = []
        filtered_labels_sublist = []

        for token, label in zip(sublist_tokens, sublist_labels):
            if token.lower() not in stop_words:
                filtered_tokens_sublist.append(token)
                filtered_labels_sublist.append(label)

        filtered_tokens.append(filtered_tokens_sublist)
        filtered_labels.append(filtered_labels_sublist)

    return filtered_tokens, filtered_labels


In [73]:
tokencount = sum(len(subarray) for subarray in training_tokens)
labelcount = sum(len(subarray) for subarray in training_labels)
tokencount, labelcount


(40000, 40000)

In [90]:
filtered_train_tokens, filtered_train_labels = remove_stopwords(training_tokens, training_labels)
filtered_valid_tokens, filtered_valid_labels = remove_stopwords(valid_tokens, valid_labels)

print(sum(len(subarray)for subarray in filtered_train_tokens))
print(sum(len(subarray) for subarray in filtered_train_labels))


print(sum(len(subarray) for subarray in filtered_valid_tokens))
print(sum(len(subarray) for subarray in filtered_valid_labels))

31497
31497
3834
3834


In [91]:

label_list, val_label_list, test_label_list = conv_label_indexes(filtered_train_labels, filtered_valid_labels, test_labels)

tokenized_datasets = tokenize_and_align_labels(filtered_train_tokens, label_list)

tokenized_val_datasets = tokenize_and_align_labels(filtered_valid_tokens, filtered_valid_labels)
tokenized_test_datasets = tokenize_and_align_labels(test_tokens, test_label_list)

tokenised_train = turn_dict_to_list_of_dict(tokenized_datasets)
tokenised_val = turn_dict_to_list_of_dict(tokenized_val_datasets)
tokenised_test = turn_dict_to_list_of_dict(tokenized_test_datasets)



In [92]:
print(label_list[5])
print(filtered_train_tokens[5])


[0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0]
['importance', 'formation', 'PFN1', '-', 'actin', 'complexes', 'regulation', 'PKC', 'corroborated', 'overexpression', '-θPFN1-', 'actin', '-', 'binding', 'defective', 'mutants', 'β', '-', 'actin', '(', 'C374S', ')', 'PFN1', '(', 'H119E', ')', ',', 'respectively', ',', 'reduced', 'coalescence', 'PKC', '-θc-SMAC', '.']


# Fine tune BERT with these tokens and labels, then evaluate against the test set

In [84]:
from transformers import AutoTokenizer, AutoModelForTokenClassification

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model2 = AutoModelForTokenClassification.from_pretrained("bert-base-uncased", num_labels=4)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [85]:
from transformers import TrainingArguments, Trainer, EarlyStoppingCallback

# Training arguments (feel free to play arround with these values)
model_name = "bert-base-uncased"
epochs = 6
batch_size = 4
learning_rate = 2e-5

args2 = TrainingArguments(
    f"BERT-finetuned-NER",
    # evaluation_strategy = "epoch", ## Instead of focusing on loss and accuracy, we will focus on the F1 score
    evaluation_strategy ='steps',
    eval_steps = 7000,
    save_total_limit = 3,
    learning_rate=learning_rate,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=epochs,
    weight_decay=0.001,
    save_steps=35000,
    metric_for_best_model = 'f1',
    load_best_model_at_end=True
)

trainer2 = Trainer(
    model2,
    args2,
    train_dataset=tokenised_train,
    eval_dataset=tokenised_val,
    data_collator = data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks = [EarlyStoppingCallback(early_stopping_patience=3)]
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False)
  warnings.warn(


In [86]:
trainer2.train()


Step,Training Loss,Validation Loss


TrainOutput(global_step=1608, training_loss=0.20144078713744434, metrics={'train_runtime': 152.5114, 'train_samples_per_second': 42.174, 'train_steps_per_second': 10.543, 'total_flos': 241438486044480.0, 'train_loss': 0.20144078713744434, 'epoch': 6.0})

In [93]:
results = calculate_results(trainer2, tokenised_test)
results

/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: [0, 2, 3, 3, 3, 3, 0, 1, 0, 0, 0] seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: [0, 0, 0, 0, 0, 2, 3, 0, 1, 0, 0, 0, 0, 0, 0, 0, 2, 3, 0, 1, 0, 0, 0, 0, 0, 0, 0, 2, 3, 3, 0, 1, 0, 0, 0, 0, 0, 2, 3, 3, 1, 0, 1, 0, 0, 0, 0, 0, 0] seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: [0, 1, 0, 0, 0, 0, 0, 0, 2, 3, 3, 0, 1, 0, 0, 0, 2, 3, 3, 0, 1, 0, 0, 0, 0, 0] seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: [1, 0, 2, 3, 3, 0] seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


{'0, 0, 0, 0, 0, 2, 3, 0, 1, 0, 0, 0, 0, 0, 0, 0, 2, 3, 0, 1, 0, 0, 0, 0, 0, 0, 0, 2, 3, 3, 0, 1, 0, 0, 0, 0, 0, 2, 3, 3, 1, 0, 1, 0, 0, 0, 0, 0, 0]': {'precision': 0.7256944444444444,
  'recall': 0.7827715355805244,
  'f1': 0.753153153153153,
  'number': 267},
 '0, 1, 0, 0, 0, 0, 0, 0, 2, 3, 3, 0, 1, 0, 0, 0, 2, 3, 3, 0, 1, 0, 0, 0, 0, 0]': {'precision': 0.6923076923076923,
  'recall': 0.7248322147651006,
  'f1': 0.7081967213114754,
  'number': 149},
 '0, 2, 3, 3, 3, 3, 0, 1, 0, 0, 0]': {'precision': 0.6754385964912281,
  'recall': 0.7182835820895522,
  'f1': 0.6962025316455696,
  'number': 536},
 '1, 0, 2, 3, 3, 0]': {'precision': 0.6884057971014492,
  'recall': 0.7364341085271318,
  'f1': 0.7116104868913857,
  'number': 129},
 'overall_precision': 0.6918402777777778,
 'overall_recall': 0.7372802960222017,
 'overall_f1': 0.7138378862516793,
 'overall_accuracy': 0.9293339347466546}

# Results of stop-word removal
With removal of stopwords, the time to train has not changed much, but the F1 score has gotten considerably worse with a score of 0.7.
The main reason for this is likely a combination of two things


1.   Stop words make up a considerable amount of many texts, so  being able to correctly 'tag' them is important. By removing them from our training data we have essentially stopped our model from 'learning' about stop words which is quite important given the task at hand.
2.   By removing stop words, we remove some of the context from the text. This will have an effect on our models ability to 'fine-tune' itself accurately to our training data



# Lemmatization / Stemming
The final pre-processing technique I wish to apply is lemmatization

In [96]:
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [99]:
lemmatizer = WordNetLemmatizer()
print(lemmatizer.lemmatize("rocks"))

rock


# Lemmatize the tokens


In [100]:
def lemmatize_tokens(tokens):
  filtered_tokens = []
  for token in tokens:
    sublist = []
    for i in token:
      sublist.append(lemmatizer.lemmatize(i))

    filtered_tokens.append(sublist)
  return filtered_tokens



In [101]:
training = dataset["train"]
valid = dataset["validation"]
test = dataset["test"]

training_tokens = training["tokens"]
training_labels = training["ner_tags"]
valid_tokens = valid["tokens"]
valid_labels = valid["ner_tags"]
test_tokens = test["tokens"]
test_labels = test["ner_tags"]




In [109]:
lemmatized_train = lemmatize_tokens(training_tokens)
lemmatized_valid = lemmatize_tokens(valid_tokens)

label_list, val_label_list, test_label_list = conv_label_indexes(training_labels, valid_labels, test_labels)


tokenized_datasets = tokenize_and_align_labels(lemmatized_train, label_list)
tokenized_val_datasets = tokenize_and_align_labels(lemmatized_valid, val_label_list)
tokenized_test_datasets = tokenize_and_align_labels(test_tokens, test_label_list)

tokenised_train = turn_dict_to_list_of_dict(tokenized_datasets)
tokenised_val = turn_dict_to_list_of_dict(tokenized_val_datasets)
tokenised_test = turn_dict_to_list_of_dict(tokenized_test_datasets)



In [110]:
from transformers import AutoTokenizer, AutoModelForTokenClassification

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model3 = AutoModelForTokenClassification.from_pretrained("bert-base-uncased", num_labels=4)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [111]:
from transformers import TrainingArguments, Trainer, EarlyStoppingCallback

# Training arguments (feel free to play arround with these values)
model_name = "bert-base-uncased"
epochs = 6
batch_size = 4
learning_rate = 2e-5

args3 = TrainingArguments(
    f"BERT-finetuned-NER",
    # evaluation_strategy = "epoch", ## Instead of focusing on loss and accuracy, we will focus on the F1 score
    evaluation_strategy ='steps',
    eval_steps = 7000,
    save_total_limit = 3,
    learning_rate=learning_rate,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=epochs,
    weight_decay=0.001,
    save_steps=35000,
    metric_for_best_model = 'f1',
    load_best_model_at_end=True
)

trainer3 = Trainer(
    model3,
    args3,
    train_dataset=tokenised_train,
    eval_dataset=tokenised_val,
    data_collator = data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks = [EarlyStoppingCallback(early_stopping_patience=3)]
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False)
  warnings.warn(


In [112]:
trainer3.train()

Step,Training Loss,Validation Loss


TrainOutput(global_step=1608, training_loss=0.16960472965714943, metrics={'train_runtime': 183.9045, 'train_samples_per_second': 34.975, 'train_steps_per_second': 8.744, 'total_flos': 277751226297792.0, 'train_loss': 0.16960472965714943, 'epoch': 6.0})

In [115]:
results = calculate_results(trainer3, tokenised_test)
results

/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: [0, 0, 0, 0, 2, 3, 3, 3, 3, 0, 1, 0, 0, 0, 0] seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: [0, 0, 0, 0, 0, 0, 0, 2, 3, 0, 1, 0, 0, 0, 0, 0, 0, 0, 2, 3, 0, 1, 0, 0, 0, 0, 0, 0, 0, 2, 3, 3, 0, 1, 0, 0, 0, 0, 0, 0, 2, 3, 3, 1, 0, 1, 0, 0, 0, 0, 0, 0] seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 3, 3, 0, 1, 0, 0, 0, 0, 2, 3, 3, 0, 1, 0, 0, 0, 0, 0] seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: [1, 0, 2, 3, 3, 0] seems not to be NE tag.
  warnings.warn('{} seems not t

{'0, 0, 0, 0, 0, 0, 0, 2, 3, 0, 1, 0, 0, 0, 0, 0, 0, 0, 2, 3, 0, 1, 0, 0, 0, 0, 0, 0, 0, 2, 3, 3, 0, 1, 0, 0, 0, 0, 0, 0, 2, 3, 3, 1, 0, 1, 0, 0, 0, 0, 0, 0]': {'precision': 0.6889632107023411,
  'recall': 0.7715355805243446,
  'f1': 0.7279151943462896,
  'number': 267},
 '0, 0, 0, 0, 2, 3, 3, 3, 3, 0, 1, 0, 0, 0, 0]': {'precision': 0.6581632653061225,
  'recall': 0.7220149253731343,
  'f1': 0.6886120996441282,
  'number': 536},
 '0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 3, 3, 0, 1, 0, 0, 0, 0, 2, 3, 3, 0, 1, 0, 0, 0, 0, 0]': {'precision': 0.6746987951807228,
  'recall': 0.7516778523489933,
  'f1': 0.7111111111111111,
  'number': 149},
 '1, 0, 2, 3, 3, 0]': {'precision': 0.6917808219178082,
  'recall': 0.7829457364341085,
  'f1': 0.7345454545454545,
  'number': 129},
 'overall_precision': 0.6722268557130943,
 'overall_recall': 0.7456059204440333,
 'overall_f1': 0.707017543859649,
 'overall_accuracy': 0.9249736881671929}